# Transformer 학습/추론

In [1]:
# 패키지 import
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 데이터로더 및 Transformer 모델 준비

In [2]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(max_length=256, batch_size=64)

dataloaders_dict = {"train":train_dl, "val":val_dl}

from utils.transformer import TransformerClassification

net = TransformerClassification(text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

net.train()

net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)

print("네트워크 설정 완료")



네트워크 설정 완료


In [3]:
criterion = nn.CrossEntropyLoss()

learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

## 훈련 및 검증 함수의 구현과 실행

In [4]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치 : ", device)
    print("---------start-----------")
    net.to(device)

    torch.backends.cudnn.benchmark = True

    for epoch in range(num_epochs):
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
            else:
                net.eval()
            epoch_loss = 0.0
            epoch_corrects = 0

            for batch in (dataloaders_dict[phase]):
                inputs = batch.Text[0].to(device)
                labels = batch.Label.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=="train"):
                    input_pad = 1
                    input_mask = (inputs != input_pad)

                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase =="train":
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [5]:
num_epochs = 10
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치 :  cuda:0
---------start-----------
Epoch 1/10 | train |  Loss: 0.5963 Acc: 0.6646
Epoch 1/10 |  val  |  Loss: 0.4431 Acc: 0.8004
Epoch 2/10 | train |  Loss: 0.4439 Acc: 0.7970
Epoch 2/10 |  val  |  Loss: 0.3887 Acc: 0.8304
Epoch 3/10 | train |  Loss: 0.4078 Acc: 0.8156
Epoch 3/10 |  val  |  Loss: 0.3744 Acc: 0.8404
Epoch 4/10 | train |  Loss: 0.3862 Acc: 0.8276
Epoch 4/10 |  val  |  Loss: 0.3597 Acc: 0.8498
Epoch 5/10 | train |  Loss: 0.3710 Acc: 0.8383
Epoch 5/10 |  val  |  Loss: 0.3512 Acc: 0.8550
Epoch 6/10 | train |  Loss: 0.3579 Acc: 0.8425
Epoch 6/10 |  val  |  Loss: 0.3549 Acc: 0.8520
Epoch 7/10 | train |  Loss: 0.3502 Acc: 0.8468
Epoch 7/10 |  val  |  Loss: 0.3434 Acc: 0.8578
Epoch 8/10 | train |  Loss: 0.3438 Acc: 0.8500
Epoch 8/10 |  val  |  Loss: 0.3522 Acc: 0.8530
Epoch 9/10 | train |  Loss: 0.3376 Acc: 0.8518
Epoch 9/10 |  val  |  Loss: 0.3563 Acc: 0.8536
Epoch 10/10 | train |  Loss: 0.3290 Acc: 0.8588
Epoch 10/10 |  val  |  Loss: 0.3686 Acc: 0.8506


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()
net_trained.to(device)

epoch_corrects = 0

for batch in test_dl:
    inputs = batch.Text[0].to(device)
    labels = batch.Label.to(device)

    with torch.set_grad_enabled(False):
        input_pad = 1
        input_mask = (inputs != input_pad)

        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)

        epoch_corrects += torch.sum(preds == labels.data)

epoch_acc = epoch_corrects.double() / len(test_dl)
print('테스트 데이터 {}개의 정답률: {:.4f}'.format(len(test_dl.dataset),epoch_acc))

테스트 데이터 25000개의 정답률: 53.6138
